In [2]:
import pandas as pd
import numpy as np
import pprint
from nltk.tokenize import TweetTokenizer
#not needed right now (maybe ever?) doing one-hot encoding next cell
#might have to use if we are doing the neutral thing
s = """
#encodes a column of parties into -1, 0, or 1
def encode_party(party_series):
    types = {"Democrat": 0, "Neutral": 0, "Republican": 0}
    output_array = []
    for party in party_series:
        if party == "Democrat":
            types["Democrat"] += 1
        elif party == "Neutral":
            types["Neutral"] += 1
        elif party == "Republican":
            types["Republican"] += 1
            
    resulting_encoding = []

    for type_ in types.keys():
        resulting_encoding.append(type_)
    resulting_encoding.sort()
    return np.asarray(resulting_encoding)
"""

In [5]:
#Hydrating data
tweets = pd.read_csv("data/ExtractedTweets.csv")

#Preprocessing data

#one-hot encoding republican and democrat
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(dtype=np.int, sparse=True, categories='auto')
parties = pd.DataFrame(onehot.fit_transform(tweets[["Party"]])\
                            .toarray(),
                            columns=["Democrat", "Republican"]) #todo add neutral column & training data
parties["Handle"] = tweets.Handle
tweets = tweets.drop(["Party"], axis=1)
encoded_data = pd.merge(parties, tweets)

corpus = []
#Concatinates all tweets in a user's recent tweets, and tokenizes the result
def concatinate_tweets(tweet_object):
    document = " "
    for tweets in tweet_object:
        for single_tweet in tweets.values[0:200]: 
            if not single_tweet.startswith("RT"):
                corpus.append(single_tweet)
                document += single_tweet + " "
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(document)


#dictionary containing the user's handle and a string representation of 200 of their tweets
handles_and_tweets = {}
for handle in encoded_data.groupby(["Handle"])["Tweet"]:
    handles_and_tweets[handle[0]] = concatinate_tweets(handle[1:])



In [55]:
#Organizing the data for training here
df = pd.DataFrame.from_dict(handles_and_tweets, orient='index')
df = df.reset_index()
df.rename(columns={'index':'Handle'}, inplace=True)
finished_product = pd.merge(parties, df, how='right')
finished_product = finished_product.set_index('Handle')
finished_product = finished_product.drop_duplicates()
X = finished_product.iloc[0:, 2:]
Y = finished_product['Democrat']

In [7]:
#tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer()
vectors=vectorizer.fit_transform(corpus)

users_tweet_history = pd.Series(X.fillna('').values.tolist()).str.join(' ')

vectorized_tweet_history = vectorizer.transform(users_tweet_history)

tfidf_vectors = pd.DataFrame(vectorized_tweet_history.toarray())
tfidf_vectors

,0,1,2,3,4,5,6,7,8,9,...,109763,109764,109765,109766,109767,109768,109769,109770,109771,109772
0,0.000000,0.015093,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.006853,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.010251,0.065042,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.007036,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.016598,0.013165,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.016511,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.016786,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.036621,0.007261,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# W2V encoding
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [63]:
#word2vec vectorization
def word2vec_vectorize(X):
    user_and_vectorized_tweets_word2vec = {}
    k = 0
    for index, tweets in X.iterrows():
        total_embeddings = np.zeros(300)
        vectorized_tweet = []
        k+=1
        for word in tweets:
            try:
                encoding_of_word = word_vectors.get_vector(word)
            except KeyError:
                encoding_of_word = np.zeros(300)
            for i, embedding in enumerate(encoding_of_word):
                total_embeddings[i] = total_embeddings[i] + embedding
        for i in range(len(total_embeddings)):
            total_embeddings[i] = total_embeddings[i] / len(tweets)

        user_and_vectorized_tweets_word2vec[index] = total_embeddings
        print(f'finished row {k}/433')
    return user_and_vectorized_tweets_word2vec
    
#handle_and_vector_word2vec = pd.DataFrame.from_dict(word2vec_vectorize(X), orient='index')

In [58]:
#Splitting up the data into train and test
from sklearn.model_selection import train_test_split

#tfidf split
X_train_tfidf, X_test_tfidf, Y_train_tfidf, Y_test_tfidf = train_test_split(tfidf_vectors, Y, test_size = 0.2)

#Word2vec split
X_train_w2v, X_test_w2v, Y_train_w2v, Y_test_w2v = train_test_split(handle_and_vector_word2vec, Y, test_size = 0.2)


In [11]:
#Training on model - MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
#Grid search CV
parameters = {'solver': ['lbfgs'],
              'max_iter': [1000,1500,2000 ],
              'alpha': 10.0 ** -np.arange(1, 10),
              'hidden_layer_sizes':np.arange(10, 15)}

tuned_MLP_classifier_w2v = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
tuned_MLP_classifier_tfidf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)


#Word2Vec evaluation
tuned_MLP_classifier_w2v.fit(X_train_w2v, Y_train_w2v)
w2v_score = tuned_MLP_classifier_w2v.score(X_test_w2v.values, Y_test_w2v)
w2v_best_params = tuned_MLP_classifier_w2v.best_params_
print(f'Word2vec score and best params:\n{w2v_score}\n{w2v_best_params}')

#tfidf evaluation
tuned_MLP_classifier_tfidf.fit(X_train_tfidf, Y_train_tfidf)
tfidf_score = tuned_MLP_classifier_tfidf.score(X_test_tfidf.values, Y_test_tfidf)
tfidf_best_params = tuned_MLP_classifier_tfidf.best_params_
print(f'TfIdf score and best params:\n{tfidf_score}\n{tfidf_best_params}')

C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Word2vec score and best params:
0.9195402298850575
{'alpha': 0.001, 'hidden_layer_sizes': 10, 'max_iter': 1500, 'solver': 'lbfgs'}
TfIdf score and best params:
0.9425287356321839
{'alpha': 0.001, 'hidden_layer_sizes': 11, 'max_iter': 2000, 'solver': 'lbfgs'}


In [12]:
#Evaluation for MLP Classifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_predictions_w2v = tuned_MLP_classifier_w2v.predict(X_test_w2v)
print("Word2Vec Confusion matrix and accuracy:")
print(confusion_matrix(Y_test_w2v.astype(int), y_predictions_w2v.astype(int)))
print(accuracy_score(Y_test_w2v.astype(int), y_predictions_w2v.astype(int)))

y_predictions_tfidf = tuned_MLP_classifier_tfidf.predict(X_test_tfidf)
print("\nTfIdf Confusion matrix and accuracy:")
print(confusion_matrix(Y_test_tfidf.astype(int), y_predictions_tfidf.astype(int)))
print(accuracy_score(Y_test_tfidf.astype(int), y_predictions_tfidf.astype(int)))

Word2Vec Confusion matrix and accuracy:
[[40  7]
 [ 0 40]]
0.9195402298850575

TfIdf Confusion matrix and accuracy:
[[46  3]
 [ 2 36]]
0.9425287356321839


In [13]:
#Training on model - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

#Grid search for hyperparameter tuning (for random forest)
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 150, num = 5)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint.pprint(random_grid)
tuned_RF_classifier_w2v = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=2, random_state=42,
                                  n_jobs = -1)
tuned_RF_classifier_tfidf = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=2, random_state=42,
                                  n_jobs = -1)
tuned_RF_classifier_w2v.fit(X_train_w2v, Y_train_w2v)
tuned_RF_classifier_tfidf.fit(X_train_tfidf, Y_train_tfidf)

{'bootstrap': [True, False],
 'max_depth': [10, 45, 80, 115, 150, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 650, 1100, 1550, 2000]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   52.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 12.6min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 650, 1100, 1550, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 45, 80, 115, 150, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [14]:
#Evaluation for RandomForest Classifier

#Evaluation:
print("Word2Vec results:")
print(f'Training Score: {tuned_RF_classifier_w2v.score(X_train_w2v, Y_train_w2v)} '
      + f'\n Validation Score: {tuned_RF_classifier_w2v.score(X_test_w2v, Y_test_w2v)}')

#Evaluation:
print("TfIdf results:")
print(f'Training Score: {tuned_RF_classifier_tfidf.score(X_train_tfidf, Y_train_tfidf)} '
      + f'\n Validation Score: {tuned_RF_classifier_tfidf.score(X_test_tfidf, Y_test_tfidf)}')

Word2Vec results:
Training Score: 1.0 
 Validation Score: 0.8160919540229885
TfIdf results:
Training Score: 1.0 
 Validation Score: 0.9310344827586207


In [48]:
#Compare to data on Congressional Meetings Speech
import glob

passages = []
labels = []
for file in glob.glob("./data/congressional_hearings_data/" +"*D*.txt"):
    inputfile = open(file)
    a = inputfile.readline()
    passages.append(a)
    labels.append(1)
    inputfile.close()

for file in glob.glob("./data/congressional_hearings_data/" +"*R*.txt"):
    inputfile = open(file)
    a = inputfile.readline()
    passages.append(a)
    labels.append(0)
    inputfile.close()


#vectorize sents with tfidf
passages_vectorized = vectorizer.transform(passages)
passages_tfidf = pd.DataFrame(passages_vectorized.toarray())

#vectorize sents with word2vec
word2vec_passages = []
for passage in passages:
    word_list = passage.split(" ")
    for word in word_list:
        total_embeddings = np.zeros(300)
        try:
            encoding_of_word = word_vectors.get_vector(word)
        except KeyError:
            encoding_of_word = np.zeros(300)
        for i, embedding in enumerate(encoding_of_word):
            total_embeddings[i] = total_embeddings[i] + embedding
    for i in range(len(total_embeddings)):
        total_embeddings[i] = total_embeddings[i] / len(word_list)
        
    word2vec_passages.append(total_embeddings)

#Evaluate on models
#TFIDF - MLP
print("TFIDF results:")
passages_tfidf_mlp = tuned_MLP_classifier_tfidf.score(passages_tfidf.values, labels)
print(f'\tMLP score of passages is: {passages_tfidf_mlp}')

#TFIDF - RF
passages_tfidf_rf = tuned_RF_classifier_tfidf.score(passages_tfidf.values, labels)
print(f'\tRF score of passages is: {passages_tfidf_rf}')

#W2V - MLP
print("Word2Vec results:")
passages_w2v_mlp = tuned_MLP_classifier_w2v.score(word2vec_passages, labels)
print(f'\tMLP Score of passages is: {passages_w2v_mlp}')

#W2V - RF
passages_w2v_rf = tuned_RF_classifier_w2v.score(word2vec_passages, labels)
print(f'\tRF score of passages is: {passages_w2v_rf}')

TFIDF results:
	MLP score of passages is: 0.5607476635514018
	RF score of passages is: 0.5204888569374551
Word2Vec results:
	MLP Score of passages is: 0.49496764917325664
	RF score of passages is: 0.49496764917325664


In [15]:
# Compare to FB data

# populate df 
fb_df = pd.read_csv("data/FacebookPosts.csv")

# one-hot encoding the parties
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(dtype=np.int, sparse=True, categories='auto')
fb_parties = pd.DataFrame(onehot.fit_transform(fb_df[["Party"]]).toarray(), columns=["Democrat", "Republican"])
fb_parties["Id"] = fb_df.Id
fb_df = fb_df.drop(["Party"], axis=1)
fb_df = pd.merge(fb_parties, fb_df)

# concatenating posts and grouping them by Id
group = fb_df.groupby(['Id'])
list(group['Text'])

def concatenate_posts(posts_object):
    document = " "
    for posts in posts_object:
        for post in posts.values[0:200]: 
            document += post.lower() + " "
    # just using the tweet tokenizer because it should tokenize fb posts just fine
    tokenizer = TweetTokenizer()
    tokenized_doc = " ".join(tokenizer.tokenize(document))
    return tokenized_doc

ids_and_posts = {}
for i in fb_df.groupby(["Id"])["Text"]:
    ids_and_posts[i[0]] = concatenate_posts(list(i[1:]))

In [74]:
def vectorize_facebook_bits(passages):
    word2vec_passages = []
    for passage in passages:
        word_list = passage.split(" ")
        for word in word_list:
            total_embeddings = np.zeros(300)
            try:
                encoding_of_word = word_vectors.get_vector(word)
            except KeyError:
                encoding_of_word = np.zeros(300)
            for i, embedding in enumerate(encoding_of_word):
                total_embeddings[i] = total_embeddings[i] + embedding
        for i in range(len(total_embeddings)):
            total_embeddings[i] = total_embeddings[i] / len(word_list)

        word2vec_passages.append(total_embeddings)
    return word2vec_passages

In [75]:
# organizing the data for testing

#Organizing the data for training here
ids_and_posts_df = pd.DataFrame.from_dict(ids_and_posts, orient='index')
ids_and_posts_df = ids_and_posts_df.reset_index()
ids_and_posts_df.rename(columns={'index':'Id'}, inplace=True)
finished_fb_df = pd.merge(fb_parties, ids_and_posts_df)
finished_fb_df = finished_fb_df.set_index('Id')
finished_fb_df = finished_fb_df.drop_duplicates()
X_tfidf = pd.DataFrame(vectorizer.transform(finished_fb_df.reset_index().drop(["Id"], axis=1).iloc[:, 2]).toarray())
X_w2v = pd.DataFrame(vectorize_facebook_bits(finished_fb_df.reset_index().drop(["Id"], axis=1).iloc[0:, 2]))
Y = finished_fb_df['Democrat']

# i think this is right so far but for some reason i had to rerun everything and now it's saying that
# modules cant be found so idk what to do
finished_fb_df

,Democrat,Republican,0
Id,,,
19787529402_10151539234804403,0,1,more regulations and stifling job creation is ...
19787529402_10151438053704403,0,1,in today  ûªs challenging budgetary environme...
307227745970624_720429937983734,0,1,i want to thank my fellow nevadans for their p...
307227745970624_702188556474539,0,1,appreciated the time of ranch manager ben patt...
307227745970624_472683686135110,0,1,veterans ' town hall in carson city on monday ...
307227745970624_469502873099078,0,1,"sat down for a chat with lori gilbert , news 1..."
307227745970624_123688024465491,0,1,"after what was a long week in washington , d ...."
15617630596_10203005477835973,0,1,inconvenient facts : president obama offers pl...
15617630596_10202753826224840,0,1,will we hear more empty promises tonight ? lik...


In [77]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

#tfidf evaluation
mlpmodelfacebook_tfidf = MLPClassifier(alpha=0.001, hidden_layer_sizes=11, max_iter=2000, solver='lbfgs').fit(X_train_tfidf, Y_train_tfidf)
facebook_score_tfidf = mlpmodelfacebook_tfidf.score(X_tfidf.values, Y.values)
predicts_tfidf = mlpmodelfacebook_tfidf.predict(X_tfidf.values)
print(f'TfIdf accuracy for facebook scores:\n{facebook_score_tfidf}')
print(confusion_matrix(Y.values.astype(int), predicts_tfidf.astype(int)))


#Word2Vec evaluation
mlpmodelfacebook_w2v = MLPClassifier(alpha=0.001, hidden_layer_sizes=10, max_iter=1500, solver='lbfgs').fit(X_train_w2v, Y_train_w2v)
mlpmodelfacebook_w2v.fit(X_train_w2v, Y_train_w2v)
facebook_score_w2v = mlpmodelfacebook_w2v.score(X_w2v.values, Y.values)
predicts_w2v = mlpmodelfacebook_w2v.predict(X_w2v.values)
print(f'Word2vec accuracy for facebook scores:\n{facebook_score_w2v}')
print(confusion_matrix(Y.values.astype(int), predicts_w2v.astype(int)))

TfIdf accuracy for facebook scores:
0.6186666666666667
[[561 324]
 [248 367]]
Word2vec accuracy for facebook scores:
0.41333333333333333
[[ 17 868]
 [ 12 603]]


In [79]:
#Training on model - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
w2v_rf_classifier = RandomForestClassifier().fit(X_train_w2v, Y_train_w2v)
tfidf_rf_classifier = RandomForestClassifier().fit(X_train_tfidf, Y_train_tfidf)

#Evaluation:
print('Random Forest Model:')
print("Word2Vec results Facebook:")
print(f'Accuracy Score: {w2v_rf_classifier.score(X_w2v.values, Y.values)} ')
#Evaluation:
print("TfIdf results Facebook:")
print(f'Accuracy Score: {tfidf_rf_classifier.score(X_tfidf.values, Y.values)} ')

C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest Model:
Word2Vec results Facebook:
Accuracy Score: 0.41333333333333333 
TfIdf results Facebook:
Accuracy Score: 0.5946666666666667 
